<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# NRPy 10-Minute Overview

## Author: Zach Etienne

## This notebook presents NRPy, a Python framework that converts intricate SymPy expressions into optimized C/C++ code for use in numerical relativity. It emphasizes the computation of 3-Christoffel symbols $\Gamma^{i}_{jk}$ and numerical derivative designations within expressions. 
[comment]: <> (added this abstract in case. omit of it's unnecessary.)

# Table of Contents

This notebook is organized as follows

1. [Part 1](#Part-1:-Why-NRPy?): Why NRPy?
1. [Part 2](#Part-2:-Constructing-3-Christoffels-$\Gamma^i_{jk}$-as-symbolic-expressions-in-terms-of-3-metric-$\gamma_{ij}$-and-derivatives): Constructing 3-Christoffels $\Gamma^i_{jk}$ as symbolic expressions in terms of 3-metric $\gamma_{ij}$ and derivatives
1. [Part 3](#Part-3:--c_codegen()-example:-Output-$\Gamma^i_{jk}$-expressions-as-optimized-C-code,-assuming-derivatives-already-specified): `c_codegen()` example: Output $\Gamma^i_{jk}$ expressions as optimized C code, assuming derivatives already specified
1. [Part 4](#Part-4:--c_codegen()-example:-Specify-numerical-derivatives-within-$\Gamma^i_{jk}$-expressions-as-finite-differences): `c_codegen()` example: Specify numerical derivatives within $\Gamma^i_{jk}$ expressions as finite differences
1. [Part 5](#Part-5:-What-next?-Navigating-the-NRPy-tutorial): What next? Navigating the NRPy tutorial

# Part 1: Why NRPy?
### \[Back to [top](#Table-of-Contents)\]

A core problem faced by numerical relativity is not that the techniques we use are particularly complex, but that *the equations numerical relativists solve are very complex*.

[Einstein notation](https://en.wikipedia.org/wiki/Einstein_notation) certainly makes the complexity more manageable, and is the most common way to perform tensorial mathematics in numerical relativity.

NRPy is built upon the idea that equations written in Einstein notation *are a form of code*, and when this code is expressed in NRPy's native (Python) language

* rank-0 tensors (scalars) are *variables*
* rank-1 tensors are *lists*
* rank-2 tensors are *lists of lists*
* ... and so forth.

Further, implied tensor summations are simply *loops*.

NRPy combines the above ideas with the incredibly powerful [SymPy](https://www.sympy.org/) computer algebra package (think: Mathematica for Python, but fully free & open source), with a custom-built code generation infrastructure for converting complex SymPy expressions directly into highly-optimized C/C++ code.

Importantly, NRPy builds on the idea of *learning by example*. The core NRPy repository contains more than 100 pedagogical and well-formatted Jupyter notebook tutorials, covering topics of core relevance to the field of numerical relativity. About 25 of these tutorials generate *complete C codes* capable of e.g., evolving the scalar wave equation, Maxwell's equations, and Einstein's equations of general relativity -- all in a variety of coordinate systems. All ~100 Jupyter notebooks are linked to from [the main NRPy tutorial table of contents.](NRPyPlus_Tutorial.ipynb)

This 10-minute overview however is designed to introduce only the very basic features of NRPy, with a core focus on the idea that *NRPy can be used to benefit any numerical relativity code*.

# Part 2: Constructing 3-Christoffels $\Gamma^i_{jk}$ as symbolic expressions in terms of 3-metric $\gamma_{ij}$ and derivatives 
### \[Back to [top](#Table-of-Contents)\]

**Problem statement**: Given a three-metric $\gamma_{ij}$, construct all 18 independent Christoffel symbols $\Gamma^i_{jk}$, which involves first derivatives of the metric. Assume that $\gamma_{ij}$ *and its derivatives* are given numerically, requiring the derivatives to be defined as symbols.

In NRPy we adopt a rigid syntax for tensors and indexed expressions involving Python lists, so that for example

* $\gamma_{ij}=$ `gammaDD[i][j]`
* $\gamma_{ij,k}=$ `gammaDD_dD[i][j][k]`

Christoffel symbols (of the first kind) are defined as ([source](https://en.wikipedia.org/wiki/Christoffel_symbols#Christoffel_symbols_of_the_first_kind)):

\begin{align}
\Gamma_{ij}^k &= \frac{1}{2} \gamma^{kl}\left(\gamma_{jl,i} + \gamma_{il,j} - \gamma_{ij,l}\right)\\
\end{align}

So first we'll define $\gamma_{ij}$ and its inverse using NRPy functions

In [1]:
import sympy as sp        # SymPy: The Python computer algebra package upon which NRPy depends
import nrpy.indexedexp as ixp  # NRPy: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

# gammaDD is a rank-2 indexed expression symmetric in indexes 0 and 1
gammaDD = ixp.declarerank2("gammaDD", symmetry="sym01", dimension=3)

# gammaUU is the inverse of gammaDD, and
# gammaDET (unused) is the determinant of gammaDD
gammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)  # inverts a 3x3 symmetric matrix

Below we interact a bit with these generated expressions, confirming they 

*Exercise to students*: Verify that $\gamma_{ij} \gamma^{ij} = 3$ using the provided data structures & Python loops. (*Hint 1*: Scroll down a couple of cells to see how the Christoffel symbols are implemented.  *Hint 2*: You will need to use SymPy's `simplify()` function to simplify the expression obtained.)

In [2]:
print("Check that gammaDD[0][1] = gammaDD[1][0]:")
print("gammaDD[0][1] = ", gammaDD[0][1])
print("gammaDD[1][0] = ", gammaDD[1][0])

print("\nOutput gammaUU[1][0] = ", gammaUU[1][0])
print("\nCheck that gammaUU[1][0] - gammaUU[0][1] = 0:")
print("gammaUU[1][0] - gammaUU[0][1] = ", gammaUU[1][0] - gammaUU[0][1])

Check that gammaDD[0][1] = gammaDD[1][0]:
gammaDD[0][1] =  gammaDD01
gammaDD[1][0] =  gammaDD01

Output gammaUU[1][0] =  (-gammaDD01*gammaDD22 + gammaDD02*gammaDD12)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)

Check that gammaUU[1][0] - gammaUU[0][1] = 0:
gammaUU[1][0] - gammaUU[0][1] =  0


Define Christoffel symbols in terms of the inverse metric and metric first derivatives:
$$
\Gamma_{ij}^k = \frac{1}{2} \gamma^{kl}\left(\gamma_{jl,i} + \gamma_{il,j} - \gamma_{ij,l}\right)
$$

In [3]:
# First define symbolic expressions for metric derivatives
gammaDD_dD = ixp.declarerank3("gammaDD_dD", symmetry="sym01", dimension=3)

# Initialize GammaUDD (3-Christoffel) to zero
GammaUDD = ixp.zerorank3(dimension=3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                GammaUDD[k][i][j] += sp.Rational(1, 2) * gammaUU[k][l] * (
                    gammaDD_dD[j][l][i] + gammaDD_dD[i][l][j] - gammaDD_dD[i][j][l])

Now let's confirm that $\Gamma^{k}_{ij} = \Gamma^k_{ji}$:

In [4]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            print(GammaUDD[i][j][k] - GammaUDD[i][k][j])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Part 3:  `c_codegen()` example: Output $\Gamma^i_{jk}$ expressions as optimized C code, assuming derivatives already specified
### \[Back to [top](#Table-of-Contents)\]

At the core of NRPy is the ability to convert SymPy expressions to highly optimized C code.

**Problem statement**: Output all 18 unique Christoffel symbols with 3 different levels of optimization supported by NRPy's core C output routine `c_codegen()`.

First we store all 18 unique Christoffel symbols, as well as their desired variable names in C, to Python lists.

In [5]:
symbols_list = []
varname_list = []
for i in range(3):
    for j in range(3):
        for k in range(j, 3):
            symbols_list += [GammaUDD[i][j][k]]
            varname_list += ["ChristoffelUDD" + str(i) + str(j) + str(k)]

Next we input these lists into NRPy's C/C++ code generation function `c_codegen()`, at three different levels of optimization.

**Optimization Level 0 (don't ever do it this way)**: Compute each Christoffel symbol independently.

In [6]:
import nrpy.c_codegen as ccg  # NRPy: Core C code output module
print(ccg.c_codegen(symbols_list, varname_list, enable_cse=False, verbose=False, enable_clang_format=True, include_braces=False))

ChristoffelUDD000 = (1.0/2.0)*gammaDD_dD000*(gammaDD11*gammaDD22 - ((gammaDD12)*(gammaDD12)))/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gammaDD01))*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - ((gammaDD02)*(gammaDD02))*gammaDD11) + (1.0/2.0)*(-gammaDD_dD001 + 2*gammaDD_dD010)*(-gammaDD01*gammaDD22 + gammaDD02*gammaDD12)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gammaDD01))*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - ((gammaDD02)*(gammaDD02))*gammaDD11) + (1.0/2.0)*(-gammaDD_dD002 + 2*gammaDD_dD020)*(gammaDD01*gammaDD12 - gammaDD02*gammaDD11)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gammaDD01))*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - ((gammaDD02)*(gammaDD02))*gammaDD11);
ChristoffelUDD001 = (1.0/2.0)*gammaDD_dD001*(gammaDD11*gammaDD22 - ((gammaDD12)*(gammaDD12)))/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gam

Notice in the above code that many expressions are recomputed time and time again. This is *incredibly inefficient*, and generally compilers won't optimize this properly. So in optimization level 1, we use common subexpression elimination.

**Optimization Level 1**: Use [common subexpression elimination (CSE)](https://en.wikipedia.org/wiki/Common_subexpression_elimination) to group common subexpressions

In [7]:
print(ccg.c_codegen(symbols_list, varname_list, enable_cse=True, verbose=False, enable_clang_format=True, include_braces=False))

const REAL tmp5 = -gammaDD_dD001 + 2*gammaDD_dD010;
const REAL tmp7 = -gammaDD_dD002 + 2*gammaDD_dD020;
const REAL tmp9 = -gammaDD_dD012 + gammaDD_dD021 + gammaDD_dD120;
const REAL tmp10 = gammaDD_dD012 - gammaDD_dD021 + gammaDD_dD120;
const REAL tmp11 = -gammaDD_dD112 + 2*gammaDD_dD121;
const REAL tmp12 = 2*gammaDD_dD011 - gammaDD_dD110;
const REAL tmp13 = gammaDD_dD012 + gammaDD_dD021 - gammaDD_dD120;
const REAL tmp14 = 2*gammaDD_dD122 - gammaDD_dD221;
const REAL tmp15 = 2*gammaDD_dD022 - gammaDD_dD220;
const REAL tmp3 = (1.0/2.0)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gammaDD01))*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - ((gammaDD02)*(gammaDD02))*gammaDD11);
const REAL tmp4 = tmp3*(gammaDD11*gammaDD22 - ((gammaDD12)*(gammaDD12)));
const REAL tmp6 = tmp3*(-gammaDD01*gammaDD22 + gammaDD02*gammaDD12);
const REAL tmp8 = tmp3*(gammaDD01*gammaDD12 - gammaDD02*gammaDD11);
const REAL tmp16 = tmp3*(-gammaDD00*gammaDD12 + gammaDD01*gammaDD02);

**Optimization Level 2**: Use CSE and take advantage of [single instruction, multiple data (SIMD) macros](https://en.wikipedia.org/wiki/Single_instruction,_multiple_data). NRPy translates these macros into assembler-level-optimized code for (x86_64) CPUs. Use case: looping over data on a numerical grid; can evaluate expressions at up to 8 gridpoints simultaneously *per CPU core*. Note that `FusedMulSubSIMD(a,b,c)` and `FusedMulAddSIMD(a,b,c)` perform *two* floating-point operations per cycle (on modern CPUs), whereas for other arithmetic operations at most one FP operation can be performed per cycle.

In [8]:
print(ccg.c_codegen(symbols_list, varname_list, enable_cse=True, enable_simd=True, verbose=False, enable_clang_format=True, include_braces=False))

static const double dbl_Integer_1 = 1.0;
MAYBE_UNUSED const REAL_SIMD_ARRAY _Integer_1 = ConstSIMD(dbl_Integer_1);

static const double dbl_Integer_2 = 2.0;
 const REAL_SIMD_ARRAY _Integer_2 = ConstSIMD(dbl_Integer_2);

static const double dbl_NegativeOne_ = -1.0;
MAYBE_UNUSED const REAL_SIMD_ARRAY _NegativeOne_ = ConstSIMD(dbl_NegativeOne_);

static const double dbl_Rational_1_2 = 1.0/2.0;
 const REAL_SIMD_ARRAY _Rational_1_2 = ConstSIMD(dbl_Rational_1_2);

const REAL_SIMD_ARRAY tmp1 = MulSIMD(_NegativeOne_, MulSIMD(gammaDD12, gammaDD12));
const REAL_SIMD_ARRAY tmp3 = MulSIMD(_NegativeOne_, MulSIMD(gammaDD01, gammaDD01));
const REAL_SIMD_ARRAY tmp4 = MulSIMD(_NegativeOne_, MulSIMD(gammaDD02, gammaDD02));
const REAL_SIMD_ARRAY tmp7 = FusedMulSubSIMD(_Integer_2, gammaDD_dD010, gammaDD_dD001);
const REAL_SIMD_ARRAY tmp9 = FusedMulSubSIMD(_Integer_2, gammaDD_dD020, gammaDD_dD002);
const REAL_SIMD_ARRAY tmp11 = AddSIMD(gammaDD_dD120, SubSIMD(gammaDD_dD021, gammaDD_dD012));
const REAL_SIMD_

# Part 4:  `c_codegen()` example: Specify numerical derivatives within $\Gamma^i_{jk}$ expressions as finite differences
### \[Back to [top](#Table-of-Contents)\]

To emphasize the infrastructure-agnostic nature of NRPy, in the above C codes we assumed that the derivatives were already computed (e.g., using finite-difference derivatives, discontinuous Galerkin methods, pseudospectral methods, finite-element methods, etc.)

In this part, we demonstrate NRPy's `c_codegen()` code, which basically prepends the above C/C++ codes with the code needed for computing arbitrary-order finite-difference derivatives.

To do this, NRPy makes the standard assumption that the underlying grid is *uniform* and that derivatives are taken with respect to functions stored at each point on the numerical grid. Appropriately, we call such functions "gridfunctions".

In the following, we assume that the input, $\gamma_{ij}$, is stored at each gridpoint. Also, we wish to store each independent component of $\Gamma^i_{jk}$ at each gridpoint as output. The below code cell also introduces the NRPy parameter interface (accessed via `NRPy_param_funcs.py`) to set the finite-differencing order to 6.

We'll use Optimization Level 1, to make the code easier to read; change `enable_SIMD=False` in the below `c_codegen()` function call to `enable_SIMD=True` to see the Optimization Level 2 version.

In [9]:
import nrpy.grid as gri     # NRPy: Functions having to do with numerical grids
import nrpy.finite_difference as fin   # NRPy: parameter interface
import nrpy.params as par   # NRPy: parameter interface

# First specify the finite-differencing order to be 6th order (all even orders > 0 supported!)
par.set_parval_from_str("finite_difference::fd_order", 6)
par.set_parval_from_str("params::Infrastructure", "ETLegacy")

# Next register 6 independent gamma_{ij} gridfunctions
gri.register_gridfunctions_for_single_rank2("gammaDD", dimension=3, symmetry="sym01")

# Then register gridfunctions for Christoffel symbols; note that varname_list is
gri.register_gridfunctions(varname_list, dimension=3, rank=3, is_basename=False)

# Finally output the finite-difference C code using c_codegen()
for i, var_name in enumerate(varname_list):
    varname_list[i] = gri.ETLegacyGridFunction.access_gf(gf_name=var_name)
print(ccg.c_codegen(symbols_list, varname_list, enable_cse=True, enable_fd_codegen=True, verbose=False, enable_clang_format=True, include_braces=False))

/*
 * NRPy-Generated GF Access/FD Code, Step 1 of 2:
 * Read gridfunction(s) from main memory and compute FD stencils as needed.
 */
const CCTK_REAL gammaDD00_i2m3 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2-3)];
const CCTK_REAL gammaDD00_i2m2 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2-2)];
const CCTK_REAL gammaDD00_i2m1 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2-1)];
const CCTK_REAL gammaDD00_i1m3 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1-3, i2)];
const CCTK_REAL gammaDD00_i1m2 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1-2, i2)];
const CCTK_REAL gammaDD00_i1m1 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1-1, i2)];
const CCTK_REAL gammaDD00_i0m3 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0-3, i1, i2)];
const CCTK_REAL gammaDD00_i0m2 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0-2, i1, i2)];
const CCTK_REAL gammaDD00_i0m1 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0-1, i1, i2)];
const CCTK_REAL gammaDD00 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)];
const CCTK_REAL gammaDD00_i0p1 = g

# Part 5: What next? Navigating the NRPy tutorial
### \[Back to [top](#Table-of-Contents)\]

As mentioned previously, NRPy is meant to be "learn by example". To that end, there are more than 100 fully documented Jupyter notebooks covering a wide variety of topics relevant to numerical relativity research and optimized algorithms for solving PDEs numerically.

So the answer to the question "*What next?*" is, naturally, "*Whatever you like!*" To continue the journey, check out [**the main NRPy tutorial table of contents**.](../home.ipynb).